In [3]:
# observability
import phoenix as px
px.launch_app()


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
import llama_index.core
llama_index.core.set_global_handler("arize_phoenix", endpoint="http://localhost:6006/v1/traces")

In [ ]:
# save default config

import pathlib
import yaml

from evidence_seeker.retrieval import RetrievalConfig

configfile = pathlib.Path("../configs/retrieval_config_default.yaml")

default_config = RetrievalConfig()
default_config_dict = default_config.model_dump()

#configfile.write_text(yaml.dump(default_config_dict))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


556

In [4]:
# Download IPCC report
!mkdir -p ../TMP
!curl https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_TS.pdf --output ../TMP/IPCC_AR6_WGI_TS.pdf

I0000 00:00:1733235587.689908   36823 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1733235588.305768   36823 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30.9M  100 30.9M    0     0  23.0M      0  0:00:01  0:00:01 --:--:-- 23.1M


In [ ]:
from evidence_seeker.retrieval import RetrievalConfig
from evidence_seeker.retrieval import DocumentRetriever
from evidence_seeker.datamodels import CheckedClaim

config = RetrievalConfig(
    ### Huggingface inference API ###
    # embed_backend_type="huggingface_inference_api",
    # embed_model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    # embed_base_url="https://router.huggingface.co/hf-inference/models/sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    # api_key_name="hf_debatelab_inference_provider",
    # bill_to="DebateLabKIT",
    ### Local model (via Huggingface API) ###
    embed_backend_type="huggingface",
    embed_model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    ### Local model via Ollama ###
    # embed_base_url="http://localhost:11434",
    # embed_backend_type="ollama",
    # embed_model_name="bge-m3"
    ### PATHS ### 
    index_hub_path="DebateLabKIT/apuz-index-es",
    index_persist_path="../TMP/APUZ/storage",
)

# TODO: Mv to package.
import os
import dotenv
dotenv.load_dotenv()
config.api_key_name in os.environ or print(
    f"Warning: Check wether you need an API token for your embedding backend ('{config.embed_backend_type}'). "
    f"If you need one, set the API token '{config.api_key_name}' as environment variable in .env file"
)

retriever = DocumentRetriever(
    config=config,
)

2025-06-03 13:20:09.251 | INFO     | evidence_seeker.retrieval.base:load_index:168 - Using index persist path: ../TMP/APUZ/storage
2025-06-03 13:20:09.252 | INFO     | evidence_seeker.retrieval.base:load_index:169 - True
2025-06-03 13:20:09.252 | INFO     | evidence_seeker.retrieval.base:load_index:193 - Loading index from disk at ../TMP/APUZ/storage/index


In [2]:
claim = CheckedClaim(
    text="Die Osterweiterung der EU hat zu einer Schwächung der EU geführt",
    negation="Die Osterweiterung der EU hat nicht zu einer Schwächung der EU geführt",
    uid="123"
)
claim = CheckedClaim(
    text="Die russische Regierung verbreitet keine Desinformation",
    negation="Die russische Regierung verbreitet keine Desinformation",
    uid="123"
)
docs = await retriever.retrieve_documents(claim)

In [3]:
docs

[Document(text='Es handelt sich zwar um eine prorussische Exklave mit russischem Militär, zugleich besteht – zumindest\nbislang – keine Verkehrsverbindung zu Russland.  Die Schließung der Grenze zur Ukraine nach Beginn der russischen Großinvasion ermöglicht es der Republik\nMoldau, durch ökonomischen Druck eine Wiedereingliederung Transnistriens in die Republik Moldau voranzutreiben.\n Beide abtrünnigen Territorien bilden zugleich Gefahrenherde für die moldauische Demokratie und die Pläne für eine europäische Integration.  Russland\nunternimmt große Kraftanstrengungen, um durch Desinformation Einfluss zu nehmen – allerdings beschränkt sich diese nicht auf die beiden separatistischen\nGebiete, sondern wirkt in der gesamten Republik Moldau.  Zugleich versucht Letztere mit Unterstützung der EU und den USA dem entgegenzuwirken und bis\n2030 tatsächlich Teil der Union zu werden.  Die Präsidentschaftswahlen und das Referendum im Herbst 2024 dürften bestimmend dafür sein.  Derzeit zeichnet si

In [6]:
from IPython.display import Markdown, display
template_added_metadata = ('- {author}: "{original_text}" [(Quelle)]({url})\n')
template_automated_metadata = ('- `{file_name}` (S. {page_label}): "{original_text}"\n')
t = ''
if "url" in docs[0].metadata.keys():
    document_template = template_added_metadata
else:
    document_template = template_automated_metadata
for doc in docs:
    t += (document_template.format(**doc.metadata))
display(Markdown(t))

- `yelizaveta_teile_2024.pdf` (S. 4): "Russland
unternimmt große Kraftanstrengungen, um durch Desinformation Einfluss zu nehmen – allerdings beschränkt sich diese nicht auf die beiden separatistischen
Gebiete, sondern wirkt in der gesamten Republik Moldau. "
- `anastasia_fragile_2024.pdf` (S. 2): "[5] Dennoch ist die Bevölkerung nach wie vor empfänglich für Desinformation, und es steht zu befürchten, dass der Kreml dies weiter
ausnutzt. "
- `anastasia_fragile_2024.pdf` (S. 1): "Desinformationskampagnen Russlands und schwindender Konsens im
Land schwächen den proeuropäischen Kurs der Regierung.
"
- `m_krieg_2024.pdf` (S. 4): "Aus internen Kreml-Dokumenten, die westliche Geheimdienste im Frühjahr 2024 veröffentlichten, ging hervor, dass im Rahmen einer laufenden
Desinformationskampagne politische Strategen und Internet-Trolle Tausende von gefälschten Nachrichtenartikeln, Beiträgen in sozialen Medien und
Kommentaren produzierten. "
- `yelizaveta_teile_2024.pdf` (S. 3): "Es waren offensichtliche Parallelen zur russischen
Berichterstattung über die Ukraine zu beobachten: Das "Regime Maia Sandus" wurde beschuldigt, die russischsprachige Bevölkerung in Gagausien zu
benachteiligen, politische Gegner zu unterdrücken und "russophob" zu sein. "
- `nadja_machtfragen_2024.pdf` (S. 4): "Russische Desinformationskampagnen und populistische Forderungen von
Șor fielen auf fruchtbaren Boden.
"
- `m_krieg_2024.pdf` (S. 5): "Support for Ukraine, Kremlin Documents Show, 8.4.2024, http://www.washingtonpost.com/
world/2024/04/08/russia-propaganda-us-ukraine.
"
- `yelizaveta_teile_2024.pdf` (S. 2): "Das Ergebnis hängt stark davon ab, inwieweit es Russland und seinen Lügen gelingt, die
moldauischen Wähler*innen mit Desinformation zu beeinflussen."
